In [1]:
import Pkg
Pkg.add("JuMP")
Pkg.add("Gurobi")
Pkg.add("MAT")
Pkg.add("Printf")
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("JLD")
Pkg.add("Plots")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Ch

In [2]:
using JuMP
using Gurobi
using MAT
using Printf
using DataFrames
using CSV
using Plots
using JLD

# High-level Settings
Zone = "NORTH" # price zone name

# read RTP price
fileln = matopen(string("./RTP_",Zone,"_2019_2021_1h.mat"))
RTP = read(fileln, string("RTP_",Zone,"_2019_2021_1h"))
close(fileln)

# read DAP price
fileln = matopen(string("./DAP_",Zone,"_2019_2021.mat"))
DAP = read(fileln, string("DAP_",Zone,"_2019_2021_Julia"))
close(fileln)


In [3]:
E = 1;  # storage energy capacity in MWh
# simulation setting
T = 24; # time steps per day
M = 1; # duaration per step in hour
N_sim = 365; # number of days
L = RTP[:,1];
Lp = L .> 0;
# Lp[1]*2

In [4]:
## RTP perferct prediction benchmark

# BES setting
P = .5; # power rating MW
eta = .9; # efficiency
e0 = .5 * E;
ef = e0;
MC = 10; # marginal discharge cost

# initialize optimization model
model = Model(Gurobi.Optimizer)
set_silent(model);# no outputs

# total discharge power
@variable(model, d[1:T], lower_bound = 0)
# total charge power
@variable(model, c[1:T], lower_bound = 0)
# total energy level
@variable(model, e[1:T], lower_bound = 0)
# @variable(model, D) # incremental cycle degradation
@variable(model, C) # total aging cost
@variable(model, R) # market revenue
@variable(model, I[1:T], Bin)

# arbitrage revenue
@constraint(model, ArbRev, M*sum(L.*(d .- c)) == R )
# degradation increments
# piece-wise linear degradation opportunity value
@constraint(model, DegCost, C == M*sum(d*MC)  )
# initial SoC evolution
@constraint(model, SoCInit, e[1] - e0 == M*(c[1]*eta - d[1]/eta) )
# rest SoC evolution
@constraint(model, SoCCont[t = 2:T], e[t] - e[t-1] == M*(c[t]*eta - d[t]/eta) )
# final energy level
@constraint(model, Enelast, e[T] >= ef )

# max discharge power, not discharge when prices are negative, a sufficient condition for avoiding simultaneous charge and discharge
# @constraint(model, DisMax[t=1:T], d[t] <= P*Lp[t] )
# # max charge power
# @constraint(model, ChrMax[t=1:T], c[t] <= P)

# charging / discharging non-conflict condition
@constraint(model, ChRating[t=1:T], c[t] <= P*(1-I[t]))
# charging / discharging non-conflict condition
@constraint(model, DchRating[t=1:T], d[t] <= P*I[t]) 

# max energy level
@constraint(model, SoCMax[t=1:T], e[t] <= E )
# warranty constraint
# @constraint(model, ETHMax, M*sum(d)/eta <= E-E*D0 )

# maximize revenue plus degradation value
@objective(model, Max, R-C);
print(model)
# ArbRev

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-13


In [5]:
## RTP perfect benchmark

# initialize
R_s = zeros(1, N_sim)
P_s = zeros(1, N_sim)
C_s = zeros(1, N_sim)

# D_s = zeros(1, N_sim) # Total discharge power to gird /MWh
C_v = zeros(24, N_sim) # daily charge
D_v = zeros(24, N_sim) # daily discharge
soc = zeros(24, N_sim) # daily SoC

@time begin
@printf("Optimization starts...\n")
    
for n = 1: N_sim       
    # update prices
    local L = RTP[:,n]
    # local Lp = L .> 0

    # update prices in constraints
    for t = 1:T
        # @printf("Step %d ", (n-1)*24+t)
        set_normalized_coefficient(ArbRev, d[t], M*L[t])
        set_normalized_coefficient(ArbRev, c[t], -M*L[t])
        # step = step+1
        # set_normalized_rhs(DisMax[t], P*Lp[t])
    end

    optimize!(model)
    
    # @printf("Step %d ", step)
    global R_s[n] = value(R)# objective_value(model);
    global C_s[n] = value(C)
    # global P_s[n] = value(R)-value(C)
    global P_s[n] = objective_value(model)


    # global D_s[n] = sum(value.(d))
    global C_v[:,n] = value.(c)*M
    global D_v[:,n] = value.(d)*M
    global soc[:,n] = value.(e)
    
    termination_status(model)
    global e0 = value.(e)[end]
        
    @printf("Day %d, Cummulative Rev %d, Cummulative Profit %d, Cummulative Cost %d, OptStatus: %s \n", n, sum(R_s), sum(P_s), sum(C_s), termination_status(model))
end
    
end
# SoC = vec(reshape(soc[:,(N_sim-365):(N_sim)],(105120,1)))
# df2 = DataFrame(SoC = SoC)
# CSV.write("SoC_eta.csv", df2)
#
# # save optimal dispatch to csv
# charge = vec(reshape(C_v[:,(N_sim-364):(N_sim)],(8760,1)))
# discharge = vec(reshape(D_v[:,(N_sim-364):(N_sim)],(8760,1)))
# SoC = vec(reshape(soc[:,(N_sim-364):(N_sim)],(8760,1)))
# df = DataFrame(Charge = charge, Discharge = discharge,SoC = SoC)
# CSV.write("dispatch.csv", df);

Optimization starts...
Day 1, Cummulative Rev 32, Cummulative Profit 14, Cummulative Cost 18, OptStatus: OPTIMAL 
Day 2, Cummulative Rev 66, Cummulative Profit 39, Cummulative Cost 27, OptStatus: OPTIMAL 
Day 3, Cummulative Rev 73, Cummulative Profit 42, Cummulative Cost 31, OptStatus: OPTIMAL 
Day 4, Cummulative Rev 82, Cummulative Profit 47, Cummulative Cost 36, OptStatus: OPTIMAL 
Day 5, Cummulative Rev 87, Cummulative Profit 47, Cummulative Cost 40, OptStatus: OPTIMAL 
Day 6, Cummulative Rev 107, Cummulative Profit 58, Cummulative Cost 49, OptStatus: OPTIMAL 
Day 7, Cummulative Rev 118, Cummulative Profit 61, Cummulative Cost 57, OptStatus: OPTIMAL 
Day 8, Cummulative Rev 161, Cummulative Profit 91, Cummulative Cost 70, OptStatus: OPTIMAL 
Day 9, Cummulative Rev 187, Cummulative Profit 108, Cummulative Cost 79, OptStatus: OPTIMAL 
Day 10, Cummulative Rev 227, Cummulative Profit 130, Cummulative Cost 97, OptStatus: OPTIMAL 
Day 11, Cummulative Rev 294, Cummulative Profit 175, Cummul